<a href="https://colab.research.google.com/github/karthikshanthp/colab/blob/main/dataframe_pivot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=174f0a77e1a580af710f755647640f9e893663b879c898057cc3aee43003303c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
data = [

    (2021, 1, "US", 5000),
    (2021, 1, "EU", 4000),
    (2021, 2, "US", 5500),
    (2021, 2, "EU", 4500),
    (2021, 3, "US", 6000),
    (2021, 3, "EU", 5000),
    (2021, 4, "US", 7000),
    (2021, 4, "EU", 6000)
        ]

In [9]:
columns = ["year", "quarter", "region", 'sales']

In [6]:
spark = SparkSession.builder.appName("dtaframe to sf").getOrCreate()

In [10]:
df = spark.createDataFrame(data, columns)

In [11]:
df.show()

+----+-------+------+-----+
|year|quarter|region|sales|
+----+-------+------+-----+
|2021|      1|    US| 5000|
|2021|      1|    EU| 4000|
|2021|      2|    US| 5500|
|2021|      2|    EU| 4500|
|2021|      3|    US| 6000|
|2021|      3|    EU| 5000|
|2021|      4|    US| 7000|
|2021|      4|    EU| 6000|
+----+-------+------+-----+



In [18]:
dfpivot = df.groupBy("year","quarter").pivot("region").avg("sales")
dfpivot.sort("year","quarter")

DataFrame[year: bigint, quarter: bigint, EU: double, US: double]

In [19]:
dfpivot.show()

+----+-------+------+------+
|year|quarter|    EU|    US|
+----+-------+------+------+
|2021|      2|4500.0|5500.0|
|2021|      1|4000.0|5000.0|
|2021|      4|6000.0|7000.0|
|2021|      3|5000.0|6000.0|
+----+-------+------+------+



In [21]:
dfpivot.filter(dfpivot["EU"]>=5000).show()

+----+-------+------+------+
|year|quarter|    EU|    US|
+----+-------+------+------+
|2021|      4|6000.0|7000.0|
|2021|      3|5000.0|6000.0|
+----+-------+------+------+



In [ ]:
# unpivotExpr = "stack(2, 'EU',EU, 'US', US) as (region,revenue)"

# unPivotDF = pivot_df.select("year","quarter", expr(unpivotExpr)).where("revenue is not null")

# unPivotDF.show()

In [23]:
unpivot = "stack(2, 'EU', EU, 'US', US) as (region, revenue)"
unpivotDF = dfpivot.select("year", "quarter", expr(unpivot)).where("revenue is not null").show()

+----+-------+------+-------+
|year|quarter|region|revenue|
+----+-------+------+-------+
|2021|      2|    EU| 4500.0|
|2021|      2|    US| 5500.0|
|2021|      1|    EU| 4000.0|
|2021|      1|    US| 5000.0|
|2021|      4|    EU| 6000.0|
|2021|      4|    US| 7000.0|
|2021|      3|    EU| 5000.0|
|2021|      3|    US| 6000.0|
+----+-------+------+-------+

